In [23]:
import pandas as pd
from dataset import get_dataframe
dataset = get_dataframe('./dataset_cleaned_emotions')

dataset.head()
dataset.shape

(15455, 41)

In [24]:
## Clean data
dataset.dropna()
dataset.columns = dataset.columns.str.replace(" ", "")
#dataset = dataset[dataset.synopsis != ''] # cannot do this
dataset = dataset[dataset.clean_description != '']
#dataset = dataset[dataset.clean_storyline != '']
dataset.shape

(15455, 41)

In [25]:
final_dataset = pd.DataFrame()
for index, row in dataset.iterrows():
    try:
        new_row = {
            "description" : row["clean_description"],
            "negative" : row["negative"],
            "sadness"  : row["sadness"],
            "trust"    : row["trust"],
            "anger"    : row["anger"],
            "positive" : row["positive"],
            "fear"     : row["fear"],
            "disgust"  : row["disgust"],
            "surprise" : row["surprise"],
            "anticipation"  : row["anticipation"],
            "joy"  : row["joy"],
            "rating" : row["rating"]
        }
        final_dataset = final_dataset.append( new_row, ignore_index=True )
    except Exception as ex:
        continue

In [26]:
final_dataset.shape

(15455, 12)

In [27]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

from sklearn import preprocessing

In [51]:
final_dataset.head()

,anger,anticipation,description,disgust,fear,joy,negative,positive,rating,sadness,surprise,trust
0,NaN,0.066667,young boy finds ancient comanche monster spiri...,0.066667,0.200000,0.066667,0.333333,0.133333,PG-13,0.066667,0.066667,NaN
1,0.125000,NaN,lifelong friends meet weekly celebrate past pr...,NaN,0.250000,NaN,0.125000,0.250000,PG-13,0.125000,0.125000,NaN
2,0.095238,0.095238,struggling bronx mechanic jaden miller uses bo...,0.047619,0.047619,0.095238,0.095238,0.190476,PG-13,0.095238,NaN,0.238095
3,0.064516,0.032258,ex con run criminal past hides ratted chaperon...,0.064516,0.064516,0.096774,0.096774,0.193548,PG-13,0.064516,0.064516,0.258065
4,0.137931,0.034483,crimean war poor planning leads british light ...,0.103448,0.103448,0.034483,0.275862,0.103448,PG-13,0.034483,0.068966,0.103448


In [52]:
final_dataset['rating'] = final_dataset['rating'].astype('category')
final_dataset['label'] = final_dataset['rating'].cat.codes
final_dataset.head

<bound method NDFrame.head of           anger  anticipation  \
0           NaN      0.066667   
1      0.125000           NaN   
2      0.095238      0.095238   
3      0.064516      0.032258   
4      0.137931      0.034483   
...         ...           ...   
15450  0.166667           NaN   
15451  0.041667      0.041667   
15452  0.117647           NaN   
15453  0.103448      0.137931   
15454       NaN      0.200000   

                                             description   disgust      fear  \
0      young boy finds ancient comanche monster spiri...  0.066667  0.200000   
1      lifelong friends meet weekly celebrate past pr...       NaN  0.250000   
2      struggling bronx mechanic jaden miller uses bo...  0.047619  0.047619   
3      ex con run criminal past hides ratted chaperon...  0.064516  0.064516   
4      crimean war poor planning leads british light ...  0.103448  0.103448   
...                                                  ...       ...       ...   
15450  drivin

In [53]:
# Word Vectorization
tfidf = TfidfVectorizer( max_df=0.8, max_features=10000)
X = final_dataset['description']
Y = final_dataset['label']

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=0 )

In [55]:
x_train_tfidf = tfidf.fit_transform(X_train)
xval_tfidf = tfidf.transform(X_test)

In [56]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [57]:
clf.fit(x_train_tfidf, Y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [58]:
y_pred = clf.predict(xval_tfidf)

In [59]:
# Evaluate the performance of the classifier
f1_score(Y_test, y_pred, average="micro")

0.4348107408605629

In [60]:
## NN

In [61]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(max_iter=300, hidden_layer_sizes=(10, 10, 10), random_state=1, verbose=True )
clf.fit(x_train_tfidf, Y_train)

Iteration 1, loss = 1.48860890
Iteration 2, loss = 1.43572938
Iteration 3, loss = 1.33107131
Iteration 4, loss = 1.21709775
Iteration 5, loss = 1.12185441
Iteration 6, loss = 1.03576673
Iteration 7, loss = 0.95769420
Iteration 8, loss = 0.87959778
Iteration 9, loss = 0.80401978
Iteration 10, loss = 0.73312707
Iteration 11, loss = 0.66963183
Iteration 12, loss = 0.61445226
Iteration 13, loss = 0.56246312
Iteration 14, loss = 0.51277419
Iteration 15, loss = 0.46696920
Iteration 16, loss = 0.42304897
Iteration 17, loss = 0.38169809
Iteration 18, loss = 0.34243812
Iteration 19, loss = 0.30775607
Iteration 20, loss = 0.27336689
Iteration 21, loss = 0.24381450
Iteration 22, loss = 0.21775728
Iteration 23, loss = 0.19527435
Iteration 24, loss = 0.17540526
Iteration 25, loss = 0.15876844
Iteration 26, loss = 0.14486548
Iteration 27, loss = 0.13182639
Iteration 28, loss = 0.12164035
Iteration 29, loss = 0.11335333
Iteration 30, loss = 0.10415419
Iteration 31, loss = 0.09928230
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=300, random_state=1,
              verbose=True)

In [62]:
y_pred = clf.predict(xval_tfidf)

In [63]:
# Evaluate the performance of the classifier
f1_score(Y_test, y_pred, average="micro")

0.36266580394694276

In [64]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test,y_pred))
print(classification_report(Y_test,y_pred))

[[ 82   0 155 120  22]
 [  0   0   3   5   3]
 [136   0 434 422 151]
 [106   0 361 508 204]
 [ 19   0  95 168  97]]
              precision    recall  f1-score   support

           0       0.24      0.22      0.23       379
           1       0.00      0.00      0.00        11
           2       0.41      0.38      0.40      1143
           3       0.42      0.43      0.42      1179
           4       0.20      0.26      0.23       379

    accuracy                           0.36      3091
   macro avg       0.25      0.26      0.25      3091
weighted avg       0.37      0.36      0.36      3091



/home/mahesh/Documents/GITHUB/GITHUB/MovieMetaPredict/models/classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mahesh/Documents/GITHUB/GITHUB/MovieMetaPredict/models/classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mahesh/Documents/GITHUB/GITHUB/MovieMetaPredict/models/classification/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in la